In [ ]:
!pip install tensorflow

In [ ]:
!pip install tensorflow mediapipe

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.1/36.1 MB 53.2 MB/s eta 0:00:00


In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, GRU
from tqdm import tqdm
import pickle

In [ ]:
import tensorflow as tf

physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    for device in physical_devices:
        try:
            tf.config.experimental.set_memory_growth(device, True)
            tf.config.set_logical_device_configuration(
                device, [tf.config.LogicalDeviceConfiguration(memory_limit=8192)]  
            )
            print(f"Configured GPU with a memory limit of 8192 MB.")
        except Exception as e:
            print(f"Error configuring GPU: {e}")
else:
    print("No GPU detected, running on CPU.")

import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, GRU


Configured GPU with a memory limit of 8192 MB.


In [ ]:
try:
    tf.keras.mixed_precision.set_global_policy('mixed_float16')
    print("Mixed precision enabled for speedup.")
except ValueError:
    print("Mixed precision not supported, running with default precision.")

def save_progress(filepath, data):
    with open(filepath, 'wb') as f:
        pickle.dump(data, f)

def load_progress(filepath):
    if os.path.exists(filepath):
        try:
            with open(filepath, 'rb') as f:
                return pickle.load(f)
        except Exception as e:
            print(f"Error loading checkpoint {filepath}: {e}")
    return []

Mixed precision enabled for speedup.


In [ ]:
def load_spatial_features(filepath, sequence_length):
    if not os.path.exists(filepath):
        print(f"File does not exist: {filepath}")
        return []

    with open(filepath, 'rb') as f:
        all_features = pickle.load(f)

    valid_batches = []
    for features in all_features:
        if len(features) < sequence_length:
            padded = np.pad(features, ((0, sequence_length - len(features)), (0, 0)), mode='constant')
            valid_batches.append(padded)
        elif len(features) > sequence_length:
            truncated = features[:sequence_length]
            valid_batches.append(truncated)
        else:
            valid_batches.append(features)

    return np.array(valid_batches)

In [ ]:
def extract_temporal_features_with_checkpoint(batches, checkpoint_path, sequence_length, total_batches):
    temporal_features = []
    start_index = len(load_progress(checkpoint_path))  

    for i in tqdm(range(start_index, total_batches), desc="Extracting Temporal Features"):
        sequence = batches[i]
        temporal_feature = model.predict(sequence[np.newaxis, :, :])  
        temporal_features.append(temporal_feature)

        if i % 100 == 0:  
            save_progress(checkpoint_path, temporal_features)

    save_progress(checkpoint_path, temporal_features)
    return temporal_features


In [ ]:
if __name__ == "__main__":
    
    base_path = 'drive/MyDrive/final_dataset/features/'
    features_dir = os.path.join(base_path, 'temporal_features/')
    os.makedirs(features_dir, exist_ok=True)

    spatial_real_path = os.path.join(base_path, 'spatial_features_real.pkl')
    spatial_fake_path = os.path.join(base_path, 'spatial_features_fake.pkl')

    real_checkpoint = os.path.join(features_dir, 'temporal_features_real.pkl')
    fake_checkpoint = os.path.join(features_dir, 'temporal_features_fake.pkl')

    sequence_length = 16

    print("Loading spatial features for real videos...")
    real_batches = load_spatial_features(spatial_real_path, sequence_length)

    print("Loading spatial features for fake videos...")
    fake_batches = load_spatial_features(spatial_fake_path, sequence_length)

    total_real_batches = len(real_batches)
    total_fake_batches = len(fake_batches)

    print("Processing real videos...")
    temporal_features_real = extract_temporal_features_with_checkpoint(
        real_batches, real_checkpoint, sequence_length, total_real_batches
    )

    print("Processing fake videos...")
    temporal_features_fake = extract_temporal_features_with_checkpoint(
        fake_batches, fake_checkpoint, sequence_length, total_fake_batches
    )

Loading spatial features for real videos...
Loading spatial features for fake videos...
Processing real videos...


Extracting Temporal Features: 100%|██████████| 967/967 [01:03<00:00, 15.20it/s]


Processing fake videos...


Extracting Temporal Features: 100%|██████████| 967/967 [01:03<00:00, 15.19it/s]
